 Environment setup  

<ul style="font-size:90%; font-family:monospace;">

<li>export PYTHONNOUSERSITE="True"</li>
<li>conda create -y -n cell2location python=3.10</li>
<li>conda activate cell2location</li>

<li>pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118</li>

<li>pip3 install scvi-tools==1.1.2</li>

<li>pip install git+https://github.com/BayraktarLab/cell2location.git#egg=cell2location[tutorials,dev]</li>
<li>conda install ipykernel</li>
<li>python -m ipykernel install --user --name=cell2location --display-name='cluster-cell2location'</li>
<li>pip install jax==0.5 <b># ONLY VERSION THAT ALLOWS CELL2LOCATION TO WORK</b></li>

<li><b># if you get a jax error</b> it is because cell2location requires deprecated versions of some of the packages, downgrade jax to 0.5</li>
<li>I also ran export PYTHONNOUSERSITE="True" before setting up the env

</ul>





QC on ST:
•	Remove spots with low diversity of expression ≤300 genes detected
•	Remove spots with low transcript abundance ≤600 total counts 
•	Remove spots with high ≥20% mitochondrial fraction (cell stress/death)
•	Manual annotation and removal of artefactual spots
Normalise:
•	Log transform counts to correct for library size differences and stabilise variance.
Identify genes with high biological variability
•	HVGs for clustering and dimensionality reduction, discard noise
Standardise by z-score transformation per gene (x - mean) / std and cap extreme values 
Dimensionality Reduction:
•	PCA: 20 components on HVGs 
•	KNN graph: 20 nearest neighbours using cosine similarity 
•	UMAP: for visualisation/sanity checks
Single-cell reference filtering for cell2location
•	Similar principles
Prepare single-cell reference for regression
•	estimate gene expression signatures for each annotated cell type within the reference dataset (accounting for batch effect also)
Train regression model (single-cell reference)
•	trains a pyro probabilistic model that learns cell-type specific gene expression signatures per cell type within reference dataset 
•	adjusts for technical noise and batch effects 
Prepare spatial AnnData for cell2location
•	Align genes between spatial and single-cell data, label batch key and raw count layer
Fit model to spatial data using single-cell reference
-	This step summarises the trained model's posterior distribution and saves it to adata_ref. After calling export_posterior, you get (among others):
-	adata_ref.varm['means_per_cluster_mu_fg'] → mean expression of each gene per cell type
-	adata_ref.varm['stds_per_cluster_mu_fg'] → standard deviations (uncertainty) for each gene/cell-type
-	adata_ref.varm['q05/q95_per_cluster_mu_fg'] → 5% / 95% quantiles of expression levels
Export posterior
•	Stores posterior distributions of cell-type abundances per spot which are comparable across samples because the model accounts for sequencing depth, batch effects, and technical variability.


Initially load data and visualise for sanity checks, then load and filter with scanpy 

In [1]:
import sys
import scanpy as sc
#import squidpy as sq #don't have this installed, will use depreciated version scanpy
import numpy as np
import anndata
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import gc
import os
import seaborn as sns

import cell2location
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs

/home/lythgo02/miniforge3/envs/cell2location/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lythgo02/miniforge3/envs/cell2location/lib/python3.11/site-packages/lightning/fabric/__init__.py:40: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


In [2]:
#local
results_folder ='/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/'
#results_folder ='/run/user/1804238067/gvfs/smb-share:server=nas-zsrv4,share=fmlab-data/group_folders/lythgo02/OV_visium/emily/cell2location/'

#on cluster
#results_folder ='/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/'

#### Create paths and names to results folders for reference regression and cell2location models

In [3]:
ref_run_name = f"{results_folder}/reference_signatures"
run_name = f"{results_folder}/cell2location_map"

Load the spot cleaned visium data:
 - read_and_qc is used to load the fresh spatial object (with images, coordinates, raw metadata, and initial QC) for each sample.
 - Then immediately after, for each sample, the script loads the pre-cleaned/“spot-cleaned” .h5ad and replaces the expression matrix in the spatial object with the cleaned one (adataSub.X = adataSc.X).
 - So the objects in slides are hybrids: spatial context and QC scaffolding from read_and_qc, but the actual expression values are the spot-cleaned data. 
 - Note no filtering based on QC metrics is carried out within this function, it just calculates the metrics 



In [ ]:

#cluster
#sp_data_folder = '/mnt/scratchc/fmlab/lythgo02/OV_visium/data/'

#local

sp_data_folder = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/data/'
#sp_data_folder = '/run/user/1804238067/gvfs/smb-share:server=nas-zsrv4,share=fmlab-data/group_folders/lythgo02/OV_visium/data/'
#Function that loads and QCs raw spatial data (filtered_feature_bc_matrix.h5)
#changes from original 
    #Keep the data sparse if possible to save memory and speed up analyses.
    #Use Scanpy’s built-in calculate_qc_metrics which supports sparse matrices, so no dense conversion is needed.
    #Calculate mitochondrial fraction from Scanpy’s output instead of manual summation.

def read_and_qc(sample_name, path=sp_data_folder + 'seq/runs/', mt_prefix='mt-'):
    """
    Reads a 10X Visium spatial dataset and calculates QC metrics.

    :param sample_name: Sample ID (string)
    :param path: Base path to data directory containing sample folders
    :return: AnnData object with spatial data and QC metrics
    """
    adata = sc.read_visium(
        os.path.join(path, str(sample_name), "outs"),
        count_file='filtered_feature_bc_matrix.h5',
        load_images=True
    )  # Load Visium data
    adata.obs['sample'] = str(sample_name)  # Annotate sample in obs
    adata.var['SYMBOL'] = adata.var_names.copy()  # Preserve original gene symbols  
    adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)  # Rename gene_ids to ENSEMBL
    adata.var_names_make_unique() # Ensure unique var_names
    #adata.var_names = adata.var['ENSEMBL']  # Use ENSEMBL IDs as var_names
    adata.var['mt'] = [gene.lower().startswith(mt_prefix.lower()) for gene in adata.var['SYMBOL']]  # mt gene mask
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)  # Calculate QC metrics incl. pct_counts_mt, changed the mitochondrial calculations to use the built-in pct_counts_mt
    adata.obs['mt_frac'] = adata.obs['pct_counts_mt'] / 100.0  # Mito fraction as ratio
    adata.obs_names = adata.obs['sample'] + '_' + adata.obs_names  # Prefix spot IDs with sample name
    adata.obs.index.name = 'spot_id'  # Name obs index as spot_id
    
    return adata




Load the samples
 - adataRaw contains the unnormalised counts
 - Overwrite with the cleaned spot counts from adataSc.X but preserve in a layer before overwriting as cell2location requires raw counts
 - adata.layers is a dictionary like attribute that allows you to store multiple versions of your expression matrix (like .X) under different names.

In [6]:
# sample IDs
sample_ids = [f'OV_{n}' for n in range(1, 7)]

slides = []  # collect per-sample hybrid AnnData objects
for s in sample_ids:
    adataRaw = read_and_qc(s, path=sp_data_folder + 'seq/runs/')  # load raw spatial data + QC
    adataSc = sc.read_h5ad(f"{sp_data_folder}annDat/{s}.h5ad")  # load spot-cleaned AnnData
    
    intersect = np.intersect1d(adataRaw.var_names, adataSc.var_names)  # intersect genes
    adataSub = adataRaw[:, intersect].copy()  # subset raw data to intersecting genes
    adataSc = adataSc[:, intersect]  # subset cleaned data
    adataSub.layers["raw_counts"] = adataSub.X.copy() #save raw counts in new layer (raw_counts) before overwriting
    adataSub.X = adataSc.X  # swap in cleaned counts
    adataSub.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)  # rename gene IDs to ENSEMBL
    adataSub.var_names = adataSub.var['ENSEMBL']  # set ENSEMBL IDs as var_names
    adataSub.var_names_make_unique() #make names unique
    adataSub.var.drop(columns='ENSEMBL', inplace=True)  # drop temp column
    
    sc.pp.calculate_qc_metrics(adataSub, inplace=True)  # recompute QC metrics (because we replaced with spot cleaned counts)
    slides.append(adataSub)  # append processed object to slides list

#get a unique name warning from ensemble id despite calling .var_names_make_unique, because adata.var_names is set to SYMBOL by default, we are using the ENSEMBL ID (I think) which we made unique 

/tmp/ipykernel_746690/3954611051.py:21: FutureWarning: Use `squidpy.read.visium` instead.
  adata = sc.read_visium(
/home/lythgo02/miniforge3/envs/cell2location/lib/python3.11/site-packages/anndata/_core/anndata.py:1794: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/lythgo02/miniforge3/envs/cell2location/lib/python3.11/site-packages/anndata/_core/anndata.py:1794: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_746690/3954611051.py:21: FutureWarning: Use `squidpy.read.visium` instead.
  adata = sc.read_visium(
/home/lythgo02/miniforge3/envs/cell2location/lib/python3.11/site-packages/anndata/_core/anndata.py:1794: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/lythgo02/miniforge3/envs/cell2location/lib/pytho

In [7]:

 # merge all slides into one AnnData object
adata = slides[0].concatenate(
    slides[1:],   #starts with first AnnData object and concatenates all others along the observations (cells/spots) axis
    batch_key="sample",  #adds new column to record sample origin
    uns_merge="unique",  #
    batch_categories=sample_ids,
    index_unique=None
)



/tmp/ipykernel_746690/1086261290.py:2: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = slides[0].concatenate(


In [8]:
adataSub.layers["raw_counts"][:5].toarray()

#raw data stored as sparse, mostly zero raw count data 


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(5, 16604), dtype=float32)

#Functions to explore andata objects 

Basic info  
 - adata — just typing the object name prints a concise summary  
 - adata.shape — (n_obs, n_vars) shape of the dataset  
 - adata.obs_keys() or adata.obs.columns — columns in the observation metadata (spots/cells)  
 - adata.var_keys() or adata.var.columns — columns in the variable metadata (genes/features)  
 - adata.uns_keys() or adata.uns.keys() — keys in unstructured annotations (e.g., images, clusters) ie stores extra info that doesn't fit into .obs or .var  

Access data  
 - adata.obs — DataFrame of spot/cell metadata  
 - adata.var — DataFrame of gene/feature metadata  
 - adata.X — expression matrix (usually sparse)  
 - adata.raw — raw (usually unnormalized) expression data if stored  
  

Quick summaries  
 - adata.obs.describe() — summary stats of observation metadata  
 - adata.var.describe() — summary stats of variable metadata  
 - adata.obs['some_column'].value_counts() — count occurrences in an obs column  
 - adata.var['some_gene_metadata_column'].unique() — unique values in a var metadata column    
    
Inspect gene names and IDs  
 - adata.var_names — list/index of gene names or IDs (strings)  
 - adata.obs_names — list/index of spot/cell barcodes or IDs (strings)  
  
Subsetting  
 - adata[obs_filter, var_filter] — subset by obs and var, can be slices, boolean arrays, or lists of names  

Visualization helpers  
 - sc.pl.spatial(adata, ...) — spatial plot (if spatial data available)


In [ ]:
print(adata.obs['sample'].unique())

#to extract and check out one sample 
adata1 = read_and_qc("OV_1", path=sp_data_folder + 'seq/runs/') #just load one sample

adata1
adata1_df = adata1.var # make dataframe of metadata
print(adata1_df.columns) #print column names of df
print(adata1_df)#

Ccl5 = adata1_df[adata1_df['SYMBOL']=='Ccl5']


columns present in metadata (adata.var_keys())  
 - **SYMBOL**:The gene symbol or gene name (e.g., "Ccl5", "Actb").
 - **mt**: Boolean flag if the gene is mitochondrial (usually important for QC).
 - **feature_types**: Type of feature, usually "Gene Expression" for genes.
 - **genome**: Reference genome build for the annotation (e.g., "mm10_FPs" for mouse).
 - **n_cells_by_counts**: Number of cells/spots where the gene is detected (count > 0).
 - **mean_counts** Average raw counts of the gene across all cells/spots.
 - **log1p_mean_counts**: Log-transformed mean counts: log(1 + mean_counts).
 - **pct_dropout_by_counts**: Percentage of cells/spots where the gene is not detected (count = 0).
 - **total_counts**: Total sum of counts of this gene across all cells/spots.
 - **log1p_total_counts**: Log-transformed total counts: log(1 + total_counts).

In [ ]:

#plot spot counts
for i, sample_name in enumerate(adata.obs['sample'].unique()): 
    slide = [sl for sl in slides if sl.obs['sample'].iloc[0] == sample_name][0] #the anndata object to plot
    with mpl.rc_context({'figure.figsize': [6, 7], 'axes.facecolor': 'white'}):   #matplotlib context manager to set plot parameters
        fig = sc.pl.spatial(
            slide,
            img_key="hires",  
            cmap='magma',  #colour map to visualise continuous values
            library_id=list(slide.uns['spatial'].keys())[0],  #specifies which spatial coordinates to use (sample_id=key)
            color=['total_counts', 'n_genes_by_counts'],   #the variables to plot
            size=1, #marker size for spots
            gene_symbols='SYMBOL',
            show=False,            # Avoid double rendering
            return_fig=True
        )
    fig.savefig(f"{results_folder}plots/qc/{sample_name}_spatial.png", dpi=300, bbox_inches="tight")

mpl.pyplot.close()

In [ ]:
#histoplots of counts
# Create subplots: one row per sample, 4 columns
fig, axs = plt.subplots(len(slides), 4, figsize=(15, 4 * len(slides) - 4))  # Create subplot grid

for i, s in enumerate(adata.obs['sample'].unique()):  # Loop over samples
    slide = adata[adata.obs['sample'] == s]  # Subset by sample
    sns.histplot(slide.obs['total_counts'], bins=50, kde=False, ax=axs[i, 0])  # Raw total counts
    axs[i, 0].set_xlim(0, adata.obs['total_counts'].max())  # Set x-axis limit to global max
    axs[i, 0].set_xlabel(f'raw_total_counts | {s}')  # Label x-axis
    sns.histplot(slide.obs['total_counts'][slide.obs['total_counts'] < 20000], bins=40, kde=False, ax=axs[i, 1])  # Total counts < 20,000
    axs[i, 1].set_xlim(0, 20000)  # Set x-axis limit to 20k
    axs[i, 1].set_xlabel(f'total_counts (<20k) | {s}')  # Label x-axis
    sns.histplot(slide.obs['n_genes_by_counts'], bins=60, kde=False, ax=axs[i, 2])  # Raw gene counts
    axs[i, 2].set_xlim(0, adata.obs['n_genes_by_counts'].max())  # Set x-axis limit to global max
    axs[i, 2].set_xlabel(f'n_genes_by_counts | {s}')  # Label x-axis
    sns.histplot(slide.obs['n_genes_by_counts'][slide.obs['n_genes_by_counts'] < 6000], bins=60, kde=False, ax=axs[i, 3])  # Gene counts < 6000
    axs[i, 3].set_xlim(0, 6000)  # Set x-axis limit to 6k
    axs[i, 3].set_xlabel(f'n_genes_by_counts (<6k) | {s}')  # Label x-axis
    plt.tight_layout()  # Fix layout
  #  plt.savefig(f"{results_folder}plots/qc/{s}qc.pdf", dpi=300, bbox_inches="tight")  # Save figure
    #plt.show()  # Show plot


Filtering
- low n spots removed - likely because we are already using the spot cleaned version
- ov3 AND ov4: low read counts means higher number of spots removed, have removed them as they will likely just be noise rather than meaningful expression information 
- ov4 also has an area of pancreas at the edge of the section, this is removed due to low read count anyway but have also manually selected these spots to make sure 


In [ ]:
#looks like spaceranger didn't check for spots covered by tissue when it was run 
#can't use this to filter out spots 
for sample in adata.obs['sample'].unique():
    in_tissue = (adata.obs['sample']==sample) &(adata.obs['in_tissue']==1)
    n = (adata.obs['sample']==sample).sum() #gives total number of spots 
    print(in_tissue.sum(), "spots out of", n)


In [ ]:

# Optional: check before filtering
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'mt_frac'], groupby='sample', stripplot=False)

#arbitrary thresholds recommended in tutorial 
#UMI - 600
#n genes detected - 300
#mitochondrial content - >20%

#just to give n spots removed per category per sample 
for sample in adata.obs['sample'].unique():
    n_genes = ((adata.obs['sample'] == sample) & (adata.obs['n_genes_by_counts'] <= 300)).sum()
    umi = ((adata.obs['sample'] == sample) & (adata.obs['total_counts'] <= 600)).sum()
    mt = ((adata.obs['sample'] == sample) & (adata.obs['mt_frac'] >= 0.2)).sum()
    print(f"{sample} -> n_genes: {n_genes}, umi: {umi}, mt: {mt}")

#low n spots removed - likely because we are already using the spot cleaned version
#ov3 AND ov4 MAIN REASON FOR REMOVAL OF SPOTS IS n genes 

#creating a copy first just for visualisation
adatacheck = adata[
    (adata.obs['n_genes_by_counts'] > 300) &
    (adata.obs['total_counts'] > 600) &
    (adata.obs['mt_frac'] < 0.2)
].copy()

sc.pp.calculate_qc_metrics(adatacheck, inplace=True)

sc.pl.violin(adatacheck, ['n_genes_by_counts', 'total_counts', 'mt_frac'], groupby='sample', stripplot=False)

plt.show()


In [ ]:

#Visualise the spots that are removed 

for i, sample_name in enumerate(adata.obs['sample'].unique()):
    # Get the slide (AnnData) corresponding to the sample
    slide = [sl for sl in slides if sl.obs['sample'].iloc[0] == sample_name][0]
    slide.obs['filtered_out'] = ['kept' if idx in adatacheck.obs_names else 'removed' for idx in slide.obs_names] #adds new obs specifying if spot is kept or filtered by comparing list of spots (names) in adatacheck with original
    
    with mpl.rc_context({'figure.figsize': [6, 7], 'axes.facecolor': 'white'}):
        fig = sc.pl.spatial(
            slide,
            img_key="hires",  # image key (H&E image)  
            library_id=list(slide.uns['spatial'].keys())[0],  # spatial coordinates key
            color='filtered_out',   # your categorical variable to color by
            palette=['orange', 'red'],  # grey = kept, red = removed
            size=0.8,            # spot size
            gene_symbols='SYMBOL',
            show=False,
            return_fig=True
        )
    fig.suptitle(f"{sample_name}")
    plt.show()
  #  fig.savefig(f"{results_folder}plots/qc/{sample_name}_filtered_spots.png", dpi=300, bbox_inches="tight")
   
#NEED TO FIGURE OUT HOW TO REMOVE DODGY SPOTS FROM ARTEFACTS IN OV4

Manual spot filters:
 - exclude ones I can't filter out by common filtering metrics
 - **XY coordinates in an AnnData object:** adata.obsm["spatial"] — an n_spots × 2 array of spot centers. Those two columns correspond to the spot center in full‑resolution image pixels, coming from tissue_positions(.csv) (pxl_col_in_fullres = x, pxl_row_in_fullres = y in Space Ranger
 - **Images & scale factors:** stored under adata.uns['spatial'][library_id]['scalefactors'] so coordinates can be mapped to the image you’re plotting  
  
The following brings up an interactive window for you to draw polygons around the spots you want to exclude:  
 - Once the vertices are the polygon are completed the spots contained within are coloured red and highlighted as excluded. Excluded spots are saved back to adata object.  
 - Go through the samples one by one (by changing lib=).  
 - Close the interactive window once finished, the spot selections are saved already.

In [ ]:
%matplotlib tk  
# Use the Tk backend to enable interactive plots in a separate window

import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For plotting
from matplotlib.widgets import PolygonSelector, Button  # For interactive polygon and button
from matplotlib.path import Path  # For geometric path operations

lib = "OV_5"  # Sample name to filter from adata
adata_sample = adata[adata.obs["sample"] == lib].copy()  # Subset adata for the selected sample
xy_full_sample = adata_sample.obsm["spatial"]  # Get full-resolution spatial coordinates

sf_lowres = adata.uns["spatial"][lib]["scalefactors"]["tissue_lowres_scalef"]  # Get scale factor for low-res image
xy_lowres_sample = xy_full_sample * sf_lowres  # Scale coordinates to match low-res image
img_lowres = adata.uns["spatial"][lib]["images"]["lowres"]  # Load low-res tissue image

selected_indices = set()  # Use a set to store selected spot indices across multiple polygons

# Callback for polygon selection
def onselect(verts):  # Called when a polygon is drawn
    path = Path(verts)  # Create a path from the polygon vertices
    inside = path.contains_points(xy_lowres_sample)  # Check which points fall inside the polygon
    new_indices = np.where(inside)[0]  # Get indices of points inside the polygon
    selected_indices.update(new_indices)  # Add new selections to the set
    print(f"Total selected spots: {len(selected_indices)}")  # Print total selected

    colors = ["red" if i in selected_indices else "orange" for i in range(len(xy_lowres_sample))]  # Update colors
    points.set_facecolor(colors)  # Apply new colors to scatter plot
    fig.canvas.draw_idle()  # Refresh the plot

    adata.obs.loc[adata_sample.obs_names, "selected_manual"] = 0  # Reset only current sample
    selected_barcodes = adata_sample.obs_names[list(selected_indices)]  # Get barcodes of selected spots
    adata.obs.loc[selected_barcodes, "selected_manual"] = 1  # Mark selected spots as 1

    selected_spot_ids = selected_barcodes  # Store selected spot IDs
    selected_coords = xy_full_sample[list(selected_indices)]  # Store selected coordinates
    print("Selected spot IDs and coordinates updated.")  # Confirm update

    global selector  # Re-enable selector for another polygon
    selector.disconnect_events()  # Disconnect old selector
    selector = PolygonSelector(ax, onselect, useblit=True)  # Create new selector


# Plot setup
fig, ax = plt.subplots(figsize=(16, 16))  # Create figure and axis
ax.imshow(img_lowres)  # Show tissue image
points = ax.scatter(xy_lowres_sample[:, 0], xy_lowres_sample[:, 1], s=6, c="orange")  # Plot spots

selector = PolygonSelector(ax, onselect, useblit=True)  # Activate polygon selector




In [ ]:

# Just to give n spots removed per category per sample
for sample in adata.obs['sample'].unique():
    n_genes = ((adata.obs['sample'] == sample) & (adata.obs['n_genes_by_counts'] <= 300)).sum()
    umi = ((adata.obs['sample'] == sample) & (adata.obs['total_counts'] <= 600)).sum()
    mt = ((adata.obs['sample'] == sample) & (adata.obs['mt_frac'] >= 0.2)).sum()
    manual = ((adata.obs['sample'] == sample) & (adata.obs['selected_manual'] == 1)).sum()
    print(f"{sample} -> n_genes: {n_genes}, umi: {umi}, mt: {mt}, manual: {manual}")


In [ ]:

# Create boolean masks for each filter
adata.obs["fail_n_genes"] = adata.obs["n_genes_by_counts"] <= 300
adata.obs["fail_umi"] = adata.obs["total_counts"] <= 600
adata.obs["fail_mt"] = adata.obs["mt_frac"] >= 0.2
adata.obs["fail_manual"] = adata.obs.get("selected_manual", False)  # assumes 1/True means filtered

# Combine into a "filtered" column
adata.obs["filtered"] = (
    adata.obs["fail_n_genes"] |
    adata.obs["fail_umi"] |
    adata.obs["fail_mt"] |
    adata.obs["fail_manual"]
)

# Make a dataframe with all info
df_filters = adata.obs[[
    "sample", 
    "n_genes_by_counts", 
    "total_counts", 
    "mt_frac", 
    "fail_n_genes", 
    "fail_umi", 
    "fail_mt", 
    "fail_manual", 
    "filtered"
]].copy()

# Save to CSV
df_filters.to_csv(f"{results_folder}emily_spot_filtering_summary.csv")
print("Saved filtering summary to spot_filtering_summary.csv")


# If you also want per-sample counts like your printout:
for sample in adata.obs['sample'].unique():
    subset = df_filters[df_filters["sample"] == sample]
    n_genes = subset["fail_n_genes"].sum()
    umi = subset["fail_umi"].sum()
    mt = subset["fail_mt"].sum()
    manual = subset["fail_manual"].sum()
    print(f"{sample} -> n_genes: {n_genes}, umi: {umi}, mt: {mt}, manual: {manual}")


In [ ]:
#actually remove the spots 

adata_filtered = adata[
    (adata.obs['n_genes_by_counts'] > 300) &
    (adata.obs['total_counts'] > 600) &
    (adata.obs['mt_frac'] < 0.2) &
    (adata.obs['selected_manual']==0)
    ].copy()

sc.pp.calculate_qc_metrics(adata_filtered, inplace=True)

#remove mitochondrial genes
adata_filtered = adata_filtered[:, ~adata_filtered.var['mt']].copy()

In [ ]:
adata_filtered.layers["raw_counts"][:5].toarray()

#still contains raw data 

In [ ]:
#adata_filtered.write(f"{results_folder}adata_filtered.h5ad")
adata_filtered.write("/tmp/adata_OV1-6_filtered.h5ad")  # save locally
!cp '/tmp/adata_OV1-6_filtered.h5ad' '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/'


Load filtered spatial data 

In [ ]:
#load
#adata_filtered = sc.read_h5ad(f"{results_folder}adata_OV1-6_filtered.h5ad")

adatatest = sc.read_h5ad(f"{results_folder}final_adata_vis_match.h5ad")

In [ ]:
adatatest.uns

In [ ]:
#Visualise the spots that are removed 

for i, sample_name in enumerate(adata.obs['sample'].unique()):
    # Get the slide (AnnData) corresponding to the sample
    slide = [sl for sl in slides if sl.obs['sample'].iloc[0] == sample_name][0]
    slide.obs['filtered_out'] = ['kept' if idx in adata_filtered.obs_names else 'removed' for idx in slide.obs_names] #adds new obs specifying if spot is kept or filtered by comparing list of spots (names) in adatacheck with original
    
    with mpl.rc_context({'figure.figsize': [6, 7], 'axes.facecolor': 'white'}):
        fig = sc.pl.spatial(
            slide,
            img_key="hires",  # image key (H&E image)  
            library_id=list(slide.uns['spatial'].keys())[0],  # spatial coordinates key
            color='filtered_out',   # your categorical variable to color by
            palette=['orange', 'red'],  # grey = kept, red = removed
            size=0.8,            # spot size
            gene_symbols='SYMBOL',
            show=False,
            return_fig=True
        )
    fig.suptitle(f"{sample_name}")
    fig.savefig(f"{results_folder}plots/qc{sample_name}_filtered_spots.png", dpi=300, bbox_inches="tight")
   

In [ ]:
#code for if you want to visualise the reads 

for i, sample_name in enumerate(adata_filtered.obs['sample'].unique()): 
    with mpl.rc_context({'figure.figsize': [6, 7],
                         'axes.facecolor': 'black'}):
        # Subset AnnData to current sample
        adata_sub = adata_filtered[adata_filtered.obs['sample'] == sample_name].copy()
        
        sc.pl.spatial(
            adata_sub,
            color=["Ccl5", "Ncam1", "eYFP"], #pick genes to plot
            img_key=None, #default tissue image
            size=1, #spot size
            vmin=0, #colour scale limits
            vmax='p99.0',
            cmap='magma', #colour map
            gene_symbols='SYMBOL', #column to use for genes 
            library_id=sample_name,  # <- specify which image to use
        )



Now we have filtered and combined version of the spatial data (still containing the raw read data as required by cell2location)  
Continue to process with scanpy for the next steps of dimensionality reduction (UMAP) and visualisation 

In [ ]:
print(type(adata_filtered))
adata_vis = adata_filtered.copy()
adata_vis.raw = adata_vis

#.raw stores unmodified version of the data, typically the original counts or normalized counts before transformations like log or scaling.

Log-transform the data to normalise expression distribution  
Find highly variable genes per sample  
Scale the data (z-score transform (x-mean.std) per gene) and cap extreme values at +-10 to avoid high expressing genes dominating  
Dimensionality rediction via: 
 - PCA - reduce to 20 principal components using only HVGs
 - KNN graph - builds neighbour graph based on cosine similarity in PCA space 
 - UMAP - non-linear embedding into 2D for visualisation 

In [ ]:
adata_vis_plt = adata_vis.copy() #plotting copy 

# Log-transform (log(data + 1))
sc.pp.log1p(adata_vis_plt)

# Find highly variable genes within each sample 
adata_vis_plt.var['highly_variable'] = False  #initially mark all genes as non-HVGs
for s in adata_vis_plt.obs['sample'].unique():

    adata_vis_plt_1 = adata_vis_plt[adata_vis_plt.obs['sample'].isin([s]), :]  #subset data to current sample
    sc.pp.highly_variable_genes(adata_vis_plt_1, min_mean=0.0125, max_mean=5, min_disp=0.5, n_top_genes=1000)  #compute HVGs 

    hvg_list = list(adata_vis_plt_1.var_names[adata_vis_plt_1.var['highly_variable']]) ##retrieve list of HVGs 
    adata_vis_plt.var.loc[hvg_list, 'highly_variable'] = True   #mark HVGs as HVGs in the whole dataset 

# Scale the data ( (data - mean) / sd )
sc.pp.scale(adata_vis_plt, max_value=10) 
# PCA, KNN construction (K nearest neighbour), UMAP
sc.tl.pca(adata_vis_plt, svd_solver='arpack', n_comps=20, use_highly_variable=True)
sc.pp.neighbors(adata_vis_plt, n_neighbors = 20, n_pcs = 20, metric='cosine')
sc.tl.umap(adata_vis_plt, min_dist = 0.3, spread = 1)

with mpl.rc_context({'figure.figsize': [8, 8],
                     'axes.facecolor': 'white'}):
    sc.pl.umap(adata_vis_plt, color=['sample'], size=30, #colour by sample
               color_map = 'RdPu', ncols = 1, #colour scheme
               legend_fontsize=10)
    fig.savefig(f"{results_folder}plots/qc/umap.png")
    #resultant umap is pretty similar to Ollie's
    

Load and process single cell data 

In [ ]:
#adata_ref = sc.read(f'/data/cast01/projects/OV_visium/data/comb_RNA.h5ad')

#sp_data_folder = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/data/'
adata_ref = sc.read(f"{sp_data_folder}merged_seurat_upk10sc.h5ad")


#note, loaded the rds object into r and converted to h5ad
    #to convert seurat object to h5ad, first convert to sce, then h5ad 

    #library(SpatialExperiment)
    #sce <- readRDS("merged_seurat_upk10sc.rds")

    #library(SingleCellExperiment)
    #sce <- as.SingleCellExperiment(sce)

    #library(zellkonverter)

    #writeH5AD(
    #   sce,
    #  file = "merged_seurat_upk10sc.h5ad")


In [ ]:
import sys
import scanpy as sc
#import squidpy as sq #don't have this installed, will use depreciated version scanpy
import numpy as np
import anndata
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import gc
import os
import seaborn as sns

import cell2location
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs

Gene filtering

In [ ]:
from cell2location.utils.filtering import filter_genes
selected = filter_genes(adata_ref, 
                        cell_count_cutoff=5, #filters out genes expressed in fewer than 5 cells
                        cell_percentage_cutoff2=0.03, #filters out genes expressed in less than 3% all cells
                        nonz_mean_cutoff=1.12) #genes detected in the number of cells between the above cutoffs are selected only when their average expression in non-zero cells is above this cutoff, recommende value

#Subsets the adata_ref AnnData object to retain only the filtered genes (those that passed the thresholds).
adata_ref = adata_ref[:, selected].copy()

adata_ref.obs.columns #list columns

In [ ]:
#check cell types 

#adata_ref.obs['Final_Annotation'].value_counts()
adata_ref.obs['newAnnot_2'].value_counts()

Updating final annotations:
 - Currently have multiple suerat cluster annotations making up cancer and other cell type
 - Cancer = clusters 1,3-5, 7, 8, 14 and 15
 - Other = clusters 2, 9 and 21  

Re-annotate:
 - cluster 21 = contaminant so exclude
 - cluster 2 = high for pancreatic genes too (likely also contamination) - exclude or name pancreatic

In [ ]:
print(adata_ref.obs['seurat_clusters'].value_counts())

#create a copy of final annotations column
adata_ref.obs['updated_annotation'] = adata_ref.obs['newAnnot_2']

#need to create new categories in updated_annotation column before the new values can be assigned to them
adata_ref.obs['updated_annotation'] = adata_ref.obs['updated_annotation'].cat.add_categories(['Contaminant', 'Tumour 1', 'Tumour 2', 'Tumour 3'])

#update the column entry based on seurat_clusters 
adata_ref.obs.loc[adata_ref.obs['seurat_clusters'] == '21', 'updated_annotation'] = 'Contaminant'
adata_ref.obs.loc[adata_ref.obs['seurat_clusters'] == '9', 'updated_annotation'] = 'Tumour 2'
adata_ref.obs.loc[adata_ref.obs['seurat_clusters'].isin(['1','3', '4', '5', '7', '8', '14', '15']), 'updated_annotation'] = 'Tumour 1'
adata_ref.obs.loc[adata_ref.obs['seurat_clusters'] == '2', 'updated_annotation'] = 'Tumour 3'


print(adata_ref.obs['updated_annotation'].value_counts())

print(adata_ref.obs.loc[adata_ref.obs['updated_annotation'] == 'Cancer', 'seurat_clusters'].value_counts())


In [ ]:
#if you want to subset to remove disinteresting types
print(set(adata_ref.obs['updated_annotation']))

exclude_list = ['Other', 'Contaminant', 'Tumour']
adata_ref = adata_ref[~adata_ref.obs['updated_annotation'].isin(exclude_list)]


Prepare for model:
 - estimate gene expression signatures for each annotated cell type within the reference dataset 

In [ ]:
import jax
import jaxlib

adata_ref.X = adata_ref.X.tocsr() #convert to csr for speed

cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                        # 10X reaction / sample / batch
                        batch_key='orig.ident', #for controlling batch effect
                        # cell type, covariate used for constructing signatures
                        labels_key='updated_annotation'
                       )



In [ ]:
from cell2location.models import RegressionModel
# instantiate the model (GPU will be picked up automatically)
mod = RegressionModel(adata_ref)

#view set up as sanity check
mod.view_anndata_setup()

Train the model
 - trains a pyro probabilistic model that learns cell-type specific gene expression signatures while controlling for background noise/technical effects
 - Epoch = one full pass through the training dataset during training
 - If max epoch isn't specified then the model defaults to max_epochs = np.min([round((20000 / n_cells) * 400), 400]) which is 634 in this case

In [ ]:
#train the model 

#test with 1 epoch first to check GPU is engaged as this will speed up vs cpu only

mod.train(max_epochs=1) #check GPU is used, should see GPU available: True (cuda), used: True

mod.train(max_epochs=400)

In [ ]:
#plot elbow curve to check number of epochs
mod.plot_history()

#
plt.savefig(f"{results_folder}training_history.png", dpi=300, bbox_inches='tight')
plt.close()

Learn the distribution of gene expression for each cell type in the reference data 

This step summarises the trained model's posterior distribution and saves it to adata_ref

After calling export_posterior, you get (among others):
    adata_ref.varm['means_per_cluster_mu_fg']
    → mean expression of each gene per cell type
    adata_ref.varm['stds_per_cluster_mu_fg']
    → standard deviations (uncertainty) for each gene/cell-type
    adata_ref.varm['q05/q95_per_cluster_mu_fg']
     → 5% / 95% quantiles of expression levels


In [ ]:

adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, #number of posterior samples to draw defult 1000
                              'batch_size': 2500}  #cells per batch to manage memory usage   
)

Save re-annotated single cell data

In [ ]:

#adata_file = f"{results_folder}upk10_sc_400_newlyAnnot.h5ad"
adata_ref.write("/tmp/upk10_sc_400_newlyAnnot.h5ad") #write to local directory if it doesn't allow you to write to cluster (h5ad files can be tricky)
#adata_file


In [ ]:
import sceasy

 View QC Plots
 Looking to see that the main density falls along the diagonal

In [ ]:
mod.plot_QC()
plt.savefig(f"{results_folder}model_qc_400.png", dpi=300, bbox_inches='tight')
#plt.close()

#currently the saved plots are two overlayed on top so need a better way to save if not working locally 

RegressionBackgroundDetectionTechPyroModel:  
 - Cell2location model performs Bayesian inference using Pyro (Pyro = deep probabilistic programming library built on PyTorch) to learn a probability distribution (via variational inference )
 - In probabilistic programming variables can represent uncertain values drawn from a probability distribution so hidden variables or parameters are learned from the data whilse quantifying uncertainty
 

In cell2location:
 - Prior: assumptions about gene expression levels and cell type abundances
 - Likelilhood: how likely observed counts are given those abundances
 - Posterior: updated estimate of cell type composition per spatial spot  
  
Uses ELBO (Evidence Lower Bound) as loss function 

Anndata object:  
 - New labels added by scvi-tools and cell2location (_indices, _scvi-batch and _scvi_labels)
 - adata.uns mod stores the trained regression model state (cell type exp sigs)
 - adata.varm stores cell-type signature matrices estimated by the model 

In [ ]:
#show to pyro probabilistic model that cell2location uses under the hood
print(mod.module)

#show elbo training log (ELBO )
print(mod.history)

#check adata object
print(mod.adata) 

In [ ]:
#save model 
mod.save(f"{results_folder}upk10sc_trained_model_400/", overwrite=True)

adata_ref.write(f"{results_folder}upk10_sc_400.h5ad")
#save anndata object with results 
#adata_file = f"{results_folder}upk10_sc_400.h5ad"

#smb doesn't like writing h5ad files so write locally and transfer
adata_ref.write('/tmp/upk10_sc_400.h5ad')
!cp '/tmp/upk10_sc_400.h5ad' '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/' #won't overwrite existing copy if already in destination folder



##Only run the cell below if loading previously saved model

In [ ]:
#can reload model later
#adata_file = f"{results_folder}/upk10_sc_400.h5ad"
#adata_ref = sc.read_h5ad(adata_file)
#mod = cell2location.models.RegressionModel.load(f"{results_folder}upk10sc_trained_model_400/", adata_ref)

Extracting reference cell types signatures as pd.DataFrame

In [ ]:
#check if mean expression sigs per cell type (posterior-derived) are stored in means_per_cluster_mu_fg
#builds list of column names for each cell type and copies into inf_aver
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}' #if data isn't there, it falls back to older dataframe 
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()


####   
inf_aver.columns = adata_ref.uns['mod']['factor_names']   #clean up names by renaming from verbose internal form to cell type name
inf_aver.iloc[0:5, 0:5]  #display top left 5x5 slice of inferred expression matrix
inf_aver.columns  #view columns 
inf_aver.loc["Foxp3",] #specific gene of interest



Spatial mapping  
 - replace default gene names in spatial adata with gene symbols 
 - find intersecting genes between spatial adata object and inf_aver from cell-type signature matrices estimated by the sc model 
 - subset both datasets by overlapping genes

In [ ]:
#replace default gene names in spatial anndata object with gene symbols stored in adata_vis.var['SYMBOL']
adata_vis.var_names = adata_vis.var['SYMBOL']
adata_vis.var_names_make_unique()

In [ ]:
#find overlapping genes and subset
intersect = np.intersect1d(adata_vis.var.SYMBOL, inf_aver.index) #intersects and sorts genes alphabetically 
intersect #contains only the genes present in both datasets

In [ ]:
adata_vis_match = adata_vis[:, intersect].copy()
inf_aver_match = inf_aver.loc[intersect, :].copy()
adata_vis_match.obs

Prepare spatial data  object for modeling:
 - Checks the gene names and cell metadata.
 - Registers which columns in adata.obs correspond to batches or other covariates.
 - Converts the AnnData to the internal format used by the model.

In [ ]:
cell2location.models.Cell2location.setup_anndata(adata=adata_vis_match, 
                                                 batch_key="sample", #batch key so you can model batch effects between samples 
                                                 layer="raw_counts") #raw counts is not stored in adata.X so have to specify the layer where raw data is stored

adata_filtered.layers.keys()


Convert sparse data to integers 
 - Cell2location expects raw counts as input
 - The model uses a Gamma-Poisson likelihood, which can only accept non-negative integers
 - Sparse matrix (current form of expression matrix) store only non-zero entries 
 - Need to convert non-zero entries to integers and assign back to adata object (in .data slot)

In [ ]:
adata_vis_match.X.data = np.asarray(adata_vis_match.X.data, dtype = 'int')

Skipping cell count step below

In [ ]:
#CURRENTLY SKIPPING THIS AS DONT KNOW WHY THERE ARE ONLY 1006

# Load Stardist cell counts
N_cells_per_sample = np.load(f"{sp_data_folder}cell2loc/stardist_cells.npy")[0:1006]
N_cells_per_sample = np.reshape(N_cells_per_sample, (1006, 1))
N_cells_per_sample = N_cells_per_sample + 1  # avoid zeros

# Convert to float32 for compatibility with Cell2location
N_cells_per_sample_new = N_cells_per_sample.astype(np.float32)

# Convert AnnData X to integers if sparse
if hasattr(adata_vis_match.X, 'data'):
    adata_vis_match.X.data = adata_vis_match.X.data.astype(int)
else:
    adata_vis_match.X = adata_vis_match.X.astype(int)

# Verify shapes and types
print("N_cells_per_sample_new shape:", N_cells_per_sample_new.shape)
print("N_cells_per_sample_new dtype:", N_cells_per_sample_new.dtype)


Initialise a cell2location model object:
 - the following links your spatial counts with reference signatures, ready for training to estimate cell type abundances per spot.
 - adata_vis_match → your spatial transcriptomics data
 - cell_state_df=inf_aver_match → your reference expression signatures from scRNA-seq, shape (n_genes, n_cell_types).
 - This tells the model what each cell type looks like in terms of gene expression.
 - detection_alpha=5 → hyperparameter controlling variance in RNA detection efficiency across spots.
 - cells per location = optional hyper-prior in Cell2location. If you don’t supply it, the model will fall back to its internal default assumption (roughly “average spot contains ~30 cells”, tuned during training).

In [ ]:
mod5cind = cell2location.models.Cell2location(
    adata_vis_match,
    cell_state_df=inf_aver_match,
    detection_alpha=5
)

#in the run cell2location per sample ipynb he uses the following with none of the stardist counts

#mod20c10_raw = cell2location.models.Cell2location(
 #   adata_vis_match,
  #  cell_state_df=inf_aver_match,
    # the expected average cell abundance: tissue-dependent
    # hyper-prior which can be estimated from paired histology:
   # N_cells_per_location=10,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection:
    #detection_alpha=20
#)

In [ ]:
mod5cind.view_anndata_setup()
#mod20c10_raw.view_anndata_setup()

In [ ]:
#free up memory by clearing other model 

#del mod5cind
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


In [ ]:

#30000 recommneded initially, based on the loss plot it looks like it stabilises after 5k so try just 10000 next time to save time

mod5cind.train(max_epochs=15000,
            # train using full data (batch_size=None)
            batch_size=None, # use all 
            train_size=1 #use all spots
           )

mod5cind

In [ ]:
mod20c10_raw.train(max_epochs=15000,
            # train using full data (batch_size=None)
            batch_size=None, # use all 
            train_size=1 #use all spots
           )

mod20c10_raw

In [ ]:
#plot loss 
mod5cind.plot_history()

#mod20c10_raw.plot_history()

Take samples from the posterior trained model and write predicted cell abundances back to spatial anndata object
 - Monte Carlo sampling - draw n samples (per spot and per cell type) from the posterior distribution to:
 - estimate mean, lower bound (q05), and upper bound (q95) for each cell type abundance per spot using the distribution learned during training  
 i.e. exporting posterior adds predicted cell abundances (and uncertanties) to dataset

In [ ]:
adata_vis_match = mod5cind.export_posterior(
    adata_vis_match, 
    sample_kwargs={'num_samples': 1000, #number of samples to draw per spot per cell type 
                   'batch_size': mod5cind.adata.n_obs} #n_obs = number of spots in dataset ie process all spots in one batch
)


In [ ]:
adata_vis_match2 = mod20c10_raw.export_posterior(
    adata_vis_match, 
    sample_kwargs={'num_samples': 1000, #number of samples to draw per spot per cell type 
                   'batch_size': mod20c10_raw.adata.n_obs} #n_obs = number of spots in dataset ie process all spots in one batch
)

To load or save model 

In [ ]:
#save
mod5cind.save(f"{results_folder}mod5cind_emily", overwrite=True)

#need to load spatial data first 
#mod5cind = cell2location.models.Cell2location.load(results_folder, adata_vis_match)


In [ ]:

#save
mod20c10_raw.save(f"{results_folder}mod20c10_emily", overwrite=True)

#load
#need to load spatial data first
#adata_vis_match = sc.read_h5ad(adata_filef"{results_folder}whereeverspatialfileis")
#mod20c10_raw = cell2location.models.Cell2location.load(results_folder, adata_vis_match)

NEED TO COMPARE THE TWO MODELS - doesn't appear to make a lot of difference when comparing plots

Check QC plot

In [ ]:

mod5cind.plot_QC()
fig = mod5cind.plot_spatial_QC_across_batches() 
fig = mod20c10_raw.plot_spatial_QC_across_batches()


Visualise cell type abundances
 - Adding 5% quantile of inferred cell abundances (confident lower bounds) = conservative estimates of cell types present
 - Cleaning column names for easier plotting.
 - Visualizing these cell types on spatial tissue slides.
 - Saving the final processed AnnData object.

In [ ]:
# Check the contents of .obsm (other matrices like means, q05)
adata_vis_match.obsm

# Check which cell types are in the mean abundance matrix
adata_vis_match.obsm['means_cell_abundance_w_sf'].columns


Define a function that converts absolute cell abundance to proportions per spatial spot - easier to interpret

In [ ]:
def convert_density_to_prop(df):
    rowSum = df.sum(axis = 1)
    rowProportions = df.div(rowSum, axis = 0)
    return rowProportions

Add cell abundance estimates to obs for plotting
  - can use the q05 = lower bound estimate i.e. v conservative to avoid false positives
  - q05 = 5% quantile of the posterior distribution, which represents that the model has a high confidence and at last 5% are present
  - or means_cell_abundance_w_sf for more realistic (but higher risk of over calling)

In [ ]:
# Add 5% quantile abundance to adata.obs
#copies estimated abundance of each cell type into .obs as separate columns indedxed by spot

#adata_vis_match.obs[adata_vis_match.uns['mod']['factor_names']] = adata_vis_match.obsm['q05_cell_abundance_w_sf']

adata_vis_match.obs[adata_vis_match.uns['mod']['factor_names']] = adata_vis_match.obsm['means_cell_abundance_w_sf']

#save
adata_vis_match.write(f"{results_folder}/final_adata_vis_match.h5ad")

#model trained with lower cell per spot count
#adata_vis_match2.obs[adata_vis_match2.uns['mod']['factor_names']] = adata_vis_match2.obsm['means_cell_abundance_w_sf']



Create table with proportions of cell types per sample

In [ ]:

overall_props_celltype = {}

for i in sample_ids:
    slide = select_slide(adata_vis_match, i)
    #Use raw counts per cell type from .obs
    celltype_counts = slide.obs[list(slide.obs.columns.intersection(
        sum(cell_groups_combined.values(), [])  # flatten all cell types
    ))]

    # Convert to proportions per spot
    celltype_props = celltype_counts.div(celltype_counts.sum(axis=1), axis=0)

    # Compute overall proportion per cell type for the sample (mean across spots)
    overall_props_celltype[i] = celltype_props.mean(axis=0)

# Convert to DataFrame for easy viewing/export
overall_props_celltype_df = pd.DataFrame(overall_props_celltype).T  # samples x cell types
overall_props_celltype_df.to_csv(f"{results_folder}cell_type_proportions.csv")
print(overall_props_celltype_df)


Plot cell types bar plot  
NOTE - MOVED OTHER PLOTTING FUNCTIONS TO ADDITIONAL SCRIPT

In [ ]:
import matplotlib.pyplot as plt

# Assuming overall_props_celltype_df is already computed
df = overall_props_celltype_df.copy()

# Create stacked bar plot
fig, ax = plt.subplots(figsize=(12, 6))

df.plot(kind='bar', stacked=True, ax=ax, colormap='tab20')

# Labels and title
ax.set_ylabel('Proportion')
ax.set_xlabel('Sample')
ax.set_title('Overall Proportion of Individual Cell Types per Sample')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # move legend outside

plt.tight_layout()
plt.show()


In [ ]:
overall_props = {}

for i in sample_ids:
    slide = select_slide(adata_vis_match, i)

    # Create group totals
    group_totals = {}
    for group_name, types in cell_groups_combined.items():
        group_totals[group_name] = slide.obs[types].sum(axis=1)

    # Convert totals → proportions (values 0–1 that sum to 1 per spot)
    group_props = convert_density_to_prop(pd.DataFrame(group_totals))

    # Store back into .obs for plotting if needed
    for group_name in group_props.columns:
        slide.obs[group_name] = group_props[group_name]

    # Compute overall proportion per cell type in this sample
    overall_props[i] = group_props.mean(axis=0)  # mean across spots

# Convert to DataFrame for easy viewing/export
overall_props_df = pd.DataFrame(overall_props).T  # samples x cell types
print(overall_props_df)


In [ ]:
import re

# Function to clean column names in a consistent way
def clean_columns(cols):
    return (
        cols.str.lower()                       # convert all column names to lowercase
            .str.replace("[()]", "", regex=True)   # remove parentheses () using regex
            .str.replace("/", "", regex=False)     # remove slashes /
            .str.replace("=", ".", regex=False)    # replace '=' with '.'
            .str.replace(r"[ -]", "_", regex=True) # replace spaces ' ' and hyphens '-' with underscores '_'
    )

adata_vis_test = adata_vis_match.copy()

# Apply cleaning function to .obs column names (metadata about cells/spots)
adata_vis_test.obs.columns = clean_columns(adata_vis_match.obs.columns)

# Apply cleaning function to .var column names (metadata about genes/features)
adata_vis_test.var.columns = clean_columns(adata_vis_match.var.columns)

# Remove technical columns that are not needed for downstream analysis
adata_vis_test.obs.drop(["_indices", "_scvi_batch", "_scvi_labels"], axis=1, inplace=True)

adata_vis_test.write(f"{results_folder}/final_adata_vis_clean.h5ad")


To save individual objects

In [ ]:

#load data 
adata_clean = sc.read_h5ad(f"{results_folder}/final_adata_vis_clean.h5ad")

In [ ]:
sample_col = "sample"   # e.g. "sample", "orig.ident", "batch"

samples = sorted(adata_clean.obs[sample_col].astype(str).unique())
outdir = "/home/lythgo02/Documents/OV_visium/split_by_sample"
os.makedirs(outdir, exist_ok=True)

for idx, smp in enumerate(samples, start=1):
    ad_sub = adata_clean[adata_clean.obs[sample_col] == smp].copy()
    out = os.path.join(outdir, f"final_adata_vis_clean_OV_{idx}.h5ad")  # _1.._6
    ad_sub.write_h5ad(out)
    print(f"Saved {smp} -> {out}")